In [57]:
import pandas as pd
import ccxt
import talib
import numpy as np

# pandas_ta 需要 Python 3.12+，当前环境是 Python 3.10
# 如果代码中实际未使用 pandas_ta，可以注释掉或使用可选导入
try:
    import pandas_ta as ta
    print("✓ pandas_ta 已加载")
except ImportError:
    print("⚠ pandas_ta 未安装（需要 Python 3.12+），但代码主要使用 talib，不影响运行")
    ta = None

⚠ pandas_ta 未安装（需要 Python 3.12+），但代码主要使用 talib，不影响运行


In [58]:
proxy_config = {
    'http': 'http://127.0.0.1:7890',
    'https': 'http://127.0.0.1:7890'
}

In [59]:
def fetch_data(symbol, timeframe='1m', limit=599):
    """
    从交易所获取K线数据
    
    参数:
        symbol (str): 交易对符号，如 'BTC/USDT'
        timeframe (str): 时间周期，如 '1m', '5m', '1h', '1d' 等
        limit (int): 获取的K线数量，默认599根
    
    返回:
        pd.DataFrame: 包含 OHLCV 数据的DataFrame，索引为时间戳
            - Open: 开盘价
            - High: 最高价
            - Low: 最低价
            - Close: 收盘价
            - Volume: 成交量
    """
    # 初始化交易所连接（使用币安）
    # 如需使用其他交易所，可修改为 ccxt.huobi(), ccxt.okx() 等
    exchange = ccxt.binance({
        'proxies': proxy_config,  # 代理配置
        'options': {
            'defaultType': 'spot'  # 现货交易
        }
    })
    
    # 获取K线数据
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    
    # 转换为DataFrame
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
    
    # 转换时间戳为日期时间格式
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    
    # 设置时间戳为索引
    df.set_index('timestamp', inplace=True)
    
    return df


In [60]:
def generate_signals(data):
    """
    计算多种技术分析指标
    
    参数:
        data (pd.DataFrame): 包含 OHLCV 数据的DataFrame
    
    返回:
        pd.DataFrame: 添加了所有技术指标的DataFrame
    
    说明:
        该函数会计算以下类别的技术指标：
        1. 趋势指标：MACD, SMA, EMA, ADX
        2. 震荡指标：RSI, Stochastic, Williams %R, CCI, MFI
        3. 波动率指标：布林带, ATR
        4. 通道指标：唐奇安通道, Keltner通道
        5. 成交量指标：OBV, 成交量移动平均
        6. 价格形态：支撑位, 阻力位
        7. 动量指标：ROC, MOM
    """
    # ========== 趋势指标 ==========
    # MACD (Moving Average Convergence Divergence) - 移动平均收敛散度
    # 用途：识别趋势变化和动量
    # macd: 快线(12日EMA) - 慢线(26日EMA)
    # macd_signal: MACD的9日EMA信号线
    # macd_hist: MACD柱状图，表示MACD与信号线的差值
    data['macd'], data['macd_signal'], data['macd_hist'] = talib.MACD(
        data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    
    # SMA (Simple Moving Average) - 简单移动平均线
    # 用途：识别趋势方向和支撑/阻力位
    # 短周期(5,10)用于短期趋势，长周期(50,200)用于长期趋势
    data['sma_5'] = talib.SMA(data['Close'], timeperiod=5)    # 5日移动平均
    data['sma_10'] = talib.SMA(data['Close'], timeperiod=10)  # 10日移动平均
    data['sma_20'] = talib.SMA(data['Close'], timeperiod=20)  # 20日移动平均
    data['sma_50'] = talib.SMA(data['Close'], timeperiod=50)  # 50日移动平均（中期趋势）
    data['sma_200'] = talib.SMA(data['Close'], timeperiod=200) # 200日移动平均（长期趋势）
    
    # EMA (Exponential Moving Average) - 指数移动平均线
    # 用途：对近期价格更敏感，能更快反应价格变化
    # EMA比SMA更平滑且对最新价格给予更高权重
    data['ema_12'] = talib.EMA(data['Close'], timeperiod=12)  # 12日EMA（常用于MACD计算）
    data['ema_26'] = talib.EMA(data['Close'], timeperiod=26)  # 26日EMA（常用于MACD计算）
    data['ema_50'] = talib.EMA(data['Close'], timeperiod=50)  # 50日EMA（中期趋势）
    data['ema_200'] = talib.EMA(data['Close'], timeperiod=200) # 200日EMA（长期趋势）
    
    # ADX (Average Directional Index) - 平均方向指数
    # 用途：衡量趋势强度，不表示方向
    # ADX > 25: 强趋势，ADX < 20: 弱趋势/震荡市
    # +DI: 正向指标，-DI: 负向指标
    data['adx'] = talib.ADX(data['High'], data['Low'], data['Close'], timeperiod=14)
    data['plus_di'] = talib.PLUS_DI(data['High'], data['Low'], data['Close'], timeperiod=14)  # 正向方向指标
    data['minus_di'] = talib.MINUS_DI(data['High'], data['Low'], data['Close'], timeperiod=14) # 负向方向指标
    
    # ========== 震荡指标 ==========
    # RSI
    data['rsi'] = talib.RSI(data['Close'], timeperiod=14)
    
    # Stochastic (随机振荡器)
    data['stoch_k'], data['stoch_d'] = talib.STOCH(
        data['High'], data['Low'], data['Close'],
        fastk_period=14, slowk_period=3, slowd_period=3)
    
    # Williams %R
    data['willr'] = talib.WILLR(data['High'], data['Low'], data['Close'], timeperiod=14)
    
    # CCI (商品通道指数)
    data['cci'] = talib.CCI(data['High'], data['Low'], data['Close'], timeperiod=14)
    
    # MFI (资金流量指标)
    data['mfi'] = talib.MFI(data['High'], data['Low'], data['Close'], data['Volume'], timeperiod=14)
    
    # ========== 波动率指标 ==========
    # Bollinger Bands
    data['bb_upper'], data['bb_middle'], data['bb_lower'] = talib.BBANDS(
        data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    # BB 宽度
    data['bb_width'] = (data['bb_upper'] - data['bb_lower']) / data['bb_middle'].replace(0, np.nan) * 100
    
    # ATR (平均真实波幅)
    data['atr'] = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=14)
    
    # ========== 通道指标 ==========
    # Donchian Channel
    data['dc_upper'] = talib.MAX(data['High'], timeperiod=20)
    data['dc_lower'] = talib.MIN(data['Low'], timeperiod=20)
    data['dc_middle'] = (data['dc_upper'] + data['dc_lower']) / 2
    
    # Keltner Channel
    data['ema_20'] = talib.EMA(data['Close'], timeperiod=20)
    data['kc_upper'] = data['ema_20'] + (2 * data['atr'])
    data['kc_lower'] = data['ema_20'] - (2 * data['atr'])
    
    # ========== 其他趋势/动量扩展 ==========
    # APO / TRIX / DEMA / TEMA / PSAR / Vortex / Aroon / VWMA
    data['apo'] = talib.APO(data['Close'], fastperiod=12, slowperiod=26)
    data['trix'] = talib.TRIX(data['Close'], timeperiod=15)
    data['dema_30'] = talib.DEMA(data['Close'], timeperiod=30)
    data['tema_30'] = talib.TEMA(data['Close'], timeperiod=30)
    data['psar'] = talib.SAR(data['High'], data['Low'], acceleration=0.02, maximum=0.2)
    # Vortex
    tr = talib.TRANGE(data['High'], data['Low'], data['Close'])
    vm_plus = (data['High'] - data['Low'].shift(1)).abs()
    vm_minus = (data['Low'] - data['High'].shift(1)).abs()
    data['vi_plus'] = vm_plus.rolling(14).sum() / tr.rolling(14).sum()
    data['vi_minus'] = vm_minus.rolling(14).sum() / tr.rolling(14).sum()
    # Aroon
    aroon_down, aroon_up = talib.AROON(data['High'], data['Low'], timeperiod=25)
    data['aroon_up'] = aroon_up
    data['aroon_down'] = aroon_down
    # VWMA
    tp = (data['High'] + data['Low'] + data['Close']) / 3.0
    data['vwma_20'] = (tp * data['Volume']).rolling(20, min_periods=20).sum() / data['Volume'].rolling(20, min_periods=20).sum()
    
    # ========== 成交量指标 ==========
    # OBV (能量潮)
    data['obv'] = talib.OBV(data['Close'], data['Volume'])
    # CMF / EMV / FI / NVI / VPT / VWAP
    mfm = ((data['Close'] - data['Low']) - (data['High'] - data['Close'])) / (data['High'] - data['Low']).replace(0, np.nan)
    mfv = mfm * data['Volume']
    data['cmf_20'] = mfv.rolling(20).sum() / data['Volume'].rolling(20).sum()
    mid_move = ((data['High'] + data['Low']) / 2.0).diff()
    br = data['High'] - data['Low']
    data['emv_14'] = (mid_move * (data['High'] - data['Low']) / br.replace(0, np.nan)).rolling(14).mean()
    data['fi_13'] = (data['Close'].diff() * data['Volume']).ewm(span=13, adjust=False).mean()
    nvi = pd.Series(index=data.index, dtype=float)
    if len(data) > 0:
        nvi.iloc[0] = 1000.0
        for i in range(1, len(data)):
            if data['Volume'].iloc[i] < data['Volume'].iloc[i-1]:
                pct = data['Close'].pct_change().iloc[i]
                nvi.iloc[i] = nvi.iloc[i-1] * (1 + pct)
            else:
                nvi.iloc[i] = nvi.iloc[i-1]
    data['nvi'] = nvi
    data['vpt'] = (data['Close'].pct_change().fillna(0) * data['Volume']).cumsum()
    cum_v = data['Volume'].cumsum().replace(0, np.nan)
    data['vwap'] = ((tp * data['Volume']).cumsum() / cum_v)
    
    # Volume SMA (成交量移动平均)
    data['volume_sma_20'] = talib.SMA(data['Volume'], timeperiod=20)
    
    # ========== 价格形态 ==========
    # 支撑位和阻力位 (使用最近的高点和低点)
    data['support'] = data['Low'].rolling(window=20).min()
    data['resistance'] = data['High'].rolling(window=20).max()
    
    # 价格相对位置
    data['price_position'] = (data['Close'] - data['support']) / (data['resistance'] - data['support'])
    
    # ========== 动量指标 ==========
    # ROC (变动率)
    data['roc'] = talib.ROC(data['Close'], timeperiod=10)
    
    # MOM (动量)
    data['mom'] = talib.MOM(data['Close'], timeperiod=10)
    
    # 为了兼容旧代码，保留旧列名
    data['macd_red'] = data['macd']
    data['macd_signal_blue'] = data['macd_signal']
    data['upper_band'] = data['bb_upper']
    data['middle_band'] = data['bb_middle']
    data['lower_band'] = data['bb_lower']
    data['upper_dc'] = data['dc_upper']
    data['lower_dc'] = data['dc_lower']
    
    return data


In [61]:
def analyze_signals(data):
    """
    分析所有技术指标并生成交易信号
    
    参数:
        data (pd.DataFrame): 包含技术指标的DataFrame（由generate_signals函数生成）
    
    返回:
        pd.DataFrame: 包含所有交易信号的DataFrame，每个信号为布尔值
        
    信号说明:
        该函数会生成80+个交易信号，包括：
        
        趋势信号:
        - macd_buy/sell: MACD买入/卖出信号
        - sma_golden_cross/death_cross: SMA金叉/死叉
        - price_above_sma_xx: 价格是否在SMA之上
        
        震荡信号:
        - rsi_overbought/oversold: RSI超买/超卖
        - stoch_bullish_cross: Stochastic金叉
        
        综合信号:
        - bullish_signals: 多头信号计数（12个信号源）
        - bearish_signals: 空头信号计数（12个信号源）
        - signal_strength: 综合信号强度（多头-空头）
        - strong_buy/buy/neutral/sell/strong_sell: 最终建议
    """
    signals = pd.DataFrame(index=data.index)
    
    # ========== MACD 信号 ==========
    signals['macd_buy'] = (data['macd'] > data['macd_signal'])
    signals['macd_sell'] = (data['macd'] < data['macd_signal'])
    signals['macd_cross_up'] = (data['macd'] > data['macd_signal']) & (data['macd'].shift(1) <= data['macd_signal'].shift(1))
    signals['macd_cross_down'] = (data['macd'] < data['macd_signal']) & (data['macd'].shift(1) >= data['macd_signal'].shift(1))
    signals['macd_hist_positive'] = data['macd_hist'] > 0
    
    # ========== 移动平均线信号 ==========
    # SMA 交叉信号
    signals['sma_golden_cross'] = (data['sma_5'] > data['sma_20']) & (data['sma_5'].shift(1) <= data['sma_20'].shift(1))
    signals['sma_death_cross'] = (data['sma_5'] < data['sma_20']) & (data['sma_5'].shift(1) >= data['sma_20'].shift(1))
    signals['sma_50_above_200'] = data['sma_50'] > data['sma_200']  # 长期趋势
    signals['price_above_sma_20'] = data['Close'] > data['sma_20']
    signals['price_above_sma_50'] = data['Close'] > data['sma_50']
    signals['price_above_sma_200'] = data['Close'] > data['sma_200']
    
    # EMA 交叉信号
    signals['ema_golden_cross'] = (data['ema_12'] > data['ema_26']) & (data['ema_12'].shift(1) <= data['ema_26'].shift(1))
    signals['ema_death_cross'] = (data['ema_12'] < data['ema_26']) & (data['ema_12'].shift(1) >= data['ema_26'].shift(1))
    signals['price_above_ema_50'] = data['Close'] > data['ema_50']
    signals['price_above_ema_200'] = data['Close'] > data['ema_200']
    
    # ========== ADX 趋势强度信号 ==========
    signals['adx_strong_trend'] = data['adx'] > 25  # 强趋势
    signals['adx_weak_trend'] = data['adx'] < 20  # 弱趋势/震荡
    signals['adx_bullish'] = (data['adx'] > 20) & (data['plus_di'] > data['minus_di'])
    signals['adx_bearish'] = (data['adx'] > 20) & (data['plus_di'] < data['minus_di'])
    
    # ========== RSI 信号 ==========
    signals['rsi_overbought'] = data['rsi'] > 70
    signals['rsi_oversold'] = data['rsi'] < 30
    signals['rsi_middle'] = (data['rsi'] > 30) & (data['rsi'] < 70)
    signals['rsi_bullish'] = (data['rsi'] > 50) & (data['rsi'] < 70)
    signals['rsi_bearish'] = (data['rsi'] < 50) & (data['rsi'] > 30)
    signals['rsi_divergence_bull'] = (data['rsi'] > data['rsi'].shift(1)) & (data['Close'] < data['Close'].shift(1))
    signals['rsi_divergence_bear'] = (data['rsi'] < data['rsi'].shift(1)) & (data['Close'] > data['Close'].shift(1))
    
    # ========== Stochastic 信号 ==========
    signals['stoch_overbought'] = (data['stoch_k'] > 80) | (data['stoch_d'] > 80)
    signals['stoch_oversold'] = (data['stoch_k'] < 20) | (data['stoch_d'] < 20)
    signals['stoch_bullish_cross'] = (data['stoch_k'] > data['stoch_d']) & (data['stoch_k'].shift(1) <= data['stoch_d'].shift(1))
    signals['stoch_bearish_cross'] = (data['stoch_k'] < data['stoch_d']) & (data['stoch_k'].shift(1) >= data['stoch_d'].shift(1))
    
    # ========== Williams %R 信号 ==========
    signals['willr_overbought'] = data['willr'] > -20
    signals['willr_oversold'] = data['willr'] < -80
    signals['willr_bullish'] = (data['willr'] > -50) & (data['willr'] < -20)
    signals['willr_bearish'] = (data['willr'] < -50) & (data['willr'] > -80)
    
    # ========== CCI 信号 ==========
    signals['cci_overbought'] = data['cci'] > 100
    signals['cci_oversold'] = data['cci'] < -100
    signals['cci_bullish'] = (data['cci'] > 0) & (data['cci'] < 100)
    signals['cci_bearish'] = (data['cci'] < 0) & (data['cci'] > -100)
    
    # ========== MFI 信号 ==========
    signals['mfi_overbought'] = data['mfi'] > 80
    signals['mfi_oversold'] = data['mfi'] < 20
    signals['mfi_bullish'] = (data['mfi'] > 50) & (data['mfi'] < 80)
    signals['mfi_bearish'] = (data['mfi'] < 50) & (data['mfi'] > 20)
    
    # ========== 布林带信号 ==========
    signals['bb_buy'] = data['Close'] < data['bb_lower']
    signals['bb_sell'] = data['Close'] > data['bb_upper']
    signals['bb_squeeze'] = (data['bb_upper'] - data['bb_lower']) < (data['bb_upper'].shift(20) - data['bb_lower'].shift(20))
    signals['bb_expansion'] = (data['bb_upper'] - data['bb_lower']) > (data['bb_upper'].shift(20) - data['bb_lower'].shift(20))
    signals['bb_price_above_middle'] = data['Close'] > data['bb_middle']
    signals['bb_price_below_middle'] = data['Close'] < data['bb_middle']
    
    # ========== ATR 波动率信号 ==========
    signals['atr_high_volatility'] = data['atr'] > data['atr'].rolling(20).mean() * 1.5
    signals['atr_low_volatility'] = data['atr'] < data['atr'].rolling(20).mean() * 0.5

    # ========== 额外趋势/动量信号 ==========
    signals['apo_positive'] = data['apo'] > 0
    signals['trix_positive'] = data['trix'] > 0
    signals['psar_long'] = data['Close'] > data['psar']
    signals['vortex_bull'] = (data['vi_plus'] > data['vi_minus']) & (data['vi_plus'].shift(1) <= data['vi_minus'].shift(1))
    signals['vortex_bear'] = (data['vi_plus'] < data['vi_minus']) & (data['vi_plus'].shift(1) >= data['vi_minus'].shift(1))
    signals['aroon_bull'] = (data['aroon_up'] > data['aroon_down']) & (data['aroon_up'].shift(1) <= data['aroon_down'].shift(1))
    signals['aroon_bear'] = (data['aroon_up'] < data['aroon_down']) & (data['aroon_up'].shift(1) >= data['aroon_down'].shift(1))
    signals['price_above_vwma'] = data['Close'] > data['vwma_20']

    # ========== Donchian Channel 信号 ==========
    signals['dc_buy'] = data['Close'] > data['dc_upper']
    signals['dc_sell'] = data['Close'] < data['dc_lower']
    signals['dc_price_above_middle'] = data['Close'] > data['dc_middle']
    signals['dc_price_below_middle'] = data['Close'] < data['dc_middle']
    
    # ========== Keltner Channel 信号 ==========
    signals['kc_buy'] = data['Close'] > data['kc_upper']
    signals['kc_sell'] = data['Close'] < data['kc_lower']
    signals['kc_price_above_middle'] = data['Close'] > data['ema_20']
    signals['kc_price_below_middle'] = data['Close'] < data['ema_20']

    # ========== 布林带扩展 ==========
    signals['bb_squeeze_low'] = data['bb_width'] < data['bb_width'].rolling(100).quantile(0.2)

    # ========== 成交量信号 ==========
    signals['volume_above_avg'] = data['Volume'] > data['volume_sma_20']
    signals['volume_spike'] = data['Volume'] > data['volume_sma_20'] * 1.5
    signals['obv_trending_up'] = data['obv'] > data['obv'].shift(5)
    signals['obv_trending_down'] = data['obv'] < data['obv'].shift(5)
    signals['volume_price_bullish'] = (data['Volume'] > data['volume_sma_20']) & (data['Close'] > data['Close'].shift(1))
    signals['volume_price_bearish'] = (data['Volume'] > data['volume_sma_20']) & (data['Close'] < data['Close'].shift(1))
    signals['cmf_positive'] = data['cmf_20'] > 0
    signals['emv_positive'] = data['emv_14'] > 0
    signals['fi_positive'] = data['fi_13'] > 0
    signals['vwap_above_ma'] = data['vwap'] > data['vwap'].rolling(50, min_periods=10).mean()

    # ========== 支撑阻力信号 ==========
    signals['price_near_support'] = (data['Close'] - data['support']) / data['support'] < 0.02  # 2%内
    signals['price_near_resistance'] = (data['resistance'] - data['Close']) / data['Close'] < 0.02  # 2%内
    signals['price_break_resistance'] = (data['Close'] > data['resistance']) & (data['Close'].shift(1) <= data['resistance'].shift(1))
    signals['price_break_support'] = (data['Close'] < data['support']) & (data['Close'].shift(1) >= data['support'].shift(1))
    signals['price_in_middle_range'] = (data['price_position'] > 0.3) & (data['price_position'] < 0.7)
    
    # ========== 动量信号 ==========
    signals['roc_positive'] = data['roc'] > 0
    signals['roc_negative'] = data['roc'] < 0
    signals['roc_increasing'] = data['roc'] > data['roc'].shift(1)
    signals['mom_positive'] = data['mom'] > 0
    signals['mom_negative'] = data['mom'] < 0
    signals['mom_increasing'] = data['mom'] > data['mom'].shift(1)
    
    # ========== 综合信号 ==========
    # 多头信号计数（加入扩展项）
    signals['bullish_signals'] = (
        signals['macd_buy'].astype(int) +
        signals['sma_golden_cross'].astype(int) +
        signals['ema_golden_cross'].astype(int) +
        signals['rsi_bullish'].astype(int) +
        signals['stoch_bullish_cross'].astype(int) +
        signals['bb_buy'].astype(int) +
        signals['dc_buy'].astype(int) +
        signals['kc_buy'].astype(int) +
        signals['volume_price_bullish'].astype(int) +
        signals['price_break_resistance'].astype(int) +
        signals['roc_positive'].astype(int) +
        signals['mom_positive'].astype(int) +
        signals['apo_positive'].astype(int) +
        signals['trix_positive'].astype(int) +
        signals['psar_long'].astype(int) +
        signals['vortex_bull'].astype(int) +
        signals['aroon_bull'].astype(int) +
        signals['price_above_vwma'].astype(int) +
        signals['cmf_positive'].astype(int) +
        signals['emv_positive'].astype(int) +
        signals['fi_positive'].astype(int) +
        signals['vwap_above_ma'].astype(int)
    )
    
    # 空头信号计数（加入扩展项）
    signals['bearish_signals'] = (
        signals['macd_sell'].astype(int) +
        signals['sma_death_cross'].astype(int) +
        signals['ema_death_cross'].astype(int) +
        signals['rsi_bearish'].astype(int) +
        signals['stoch_bearish_cross'].astype(int) +
        signals['bb_sell'].astype(int) +
        signals['dc_sell'].astype(int) +
        signals['kc_sell'].astype(int) +
        signals['volume_price_bearish'].astype(int) +
        signals['price_break_support'].astype(int) +
        signals['roc_negative'].astype(int) +
        signals['mom_negative'].astype(int) +
        signals['vortex_bear'].astype(int) +
        signals['aroon_bear'].astype(int)
    )
    
    # 综合信号强度
    signals['signal_strength'] = signals['bullish_signals'] - signals['bearish_signals']
    signals['strong_buy'] = signals['signal_strength'] >= 5
    signals['buy'] = (signals['signal_strength'] >= 3) & (signals['signal_strength'] < 5)
    signals['neutral'] = (signals['signal_strength'] > -3) & (signals['signal_strength'] < 3)
    signals['sell'] = (signals['signal_strength'] <= -3) & (signals['signal_strength'] > -5)
    signals['strong_sell'] = signals['signal_strength'] <= -5
    
    # 为了兼容旧代码
    signals['macd_red'] = signals['macd_buy']
    signals['bollinger_buy'] = signals['bb_buy']
    signals['bollinger_sell'] = signals['bb_sell']
    signals['bollinger_middle_line_buy'] = signals['bb_price_above_middle']
    signals['bollinger_middle_line_sell'] = signals['bb_price_below_middle']
    signals['donchian_buy'] = signals['dc_buy']
    signals['donchian_sell'] = signals['dc_sell']
    signals['donchian_buy_middle'] = signals['dc_price_above_middle']
    signals['donchian_sell_middle'] = signals['dc_price_below_middle']
    
    # 返回信号DataFrame，不包含显示逻辑
    return signals


In [62]:
def display_analysis_report(signals, data, show_details=True):
    """
    显示完整的技术分析报告
    
    参数:
        signals (pd.DataFrame): 由 analyze_signals() 函数生成的信号DataFrame
        data (pd.DataFrame): 包含指标数据的DataFrame
        show_details (bool): 是否显示详细信息，默认True
    
    返回:
        dict: 包含报告摘要的字典
            - timestamp: 时间戳
            - price: 当前价格
            - recommendation: 综合建议
            - signal_strength: 信号强度
            - bullish_count: 多头信号数
            - bearish_count: 空头信号数
    """
    # 获取最新数据
    last_signals = signals.iloc[-1]
    last_data = data.iloc[-1]
    current_time = signals.index[-1]
    current_price = last_data['Close']
    
    # 计算信号统计
    bullish_count = int(last_signals['bullish_signals'])
    bearish_count = int(last_signals['bearish_signals'])
    signal_strength = int(last_signals['signal_strength'])
    
    # 打印报告头部
    print("=" * 80)
    print("📊 技术分析信号汇总 - 综合报告")
    print("=" * 80)
    print(f"⏰ 分析时间: {current_time}")
    print(f"💰 当前价格: ${current_price:,.2f}")
    print(f"📈 多头信号数: {bullish_count} | 📉 空头信号数: {bearish_count}")
    print(f"⚡ 综合信号强度: {signal_strength}")
    
    # 综合建议判断
    if last_signals['strong_buy']:
        recommendation = "🟢🟢🟢 强烈买入"
    elif last_signals['buy']:
        recommendation = "🟢🟢 买入"
    elif last_signals['strong_sell']:
        recommendation = "🔴🔴🔴 强烈卖出"
    elif last_signals['sell']:
        recommendation = "🔴🔴 卖出"
    else:
        recommendation = "⚪ 中性/观望"
    
    print(f"🎯 综合建议: {recommendation}")
    print("=" * 80)
    
    if not show_details:
        return {
            'timestamp': current_time,
            'price': current_price,
            'recommendation': recommendation,
            'signal_strength': signal_strength,
            'bullish_count': bullish_count,
            'bearish_count': bearish_count
        }
    
    # ========== 趋势指标详情 ==========
    print("\n📈 趋势指标:")
    print("-" * 80)
    trend_signals = {
        'MACD 买入': last_signals['macd_buy'],
        'MACD 金叉': last_signals['macd_cross_up'],
        'SMA 金叉 (5>20)': last_signals['sma_golden_cross'],
        'EMA 金叉 (12>26)': last_signals['ema_golden_cross'],
        '价格 > SMA50': last_signals['price_above_sma_50'],
        '价格 > SMA200': last_signals['price_above_sma_200'],
        'SMA50 > SMA200 (长期趋势)': last_signals['sma_50_above_200'],
        'ADX 强趋势 (>25)': last_signals['adx_strong_trend'],
        'ADX 看涨': last_signals['adx_bullish']
    }
    for signal_name, value in trend_signals.items():
        status = "✅" if value else "❌"
        print(f"  {status} {signal_name:<45}")
    
    # ========== 震荡指标详情 ==========
    print("\n📊 震荡指标:")
    print("-" * 80)
    print(f"  RSI: {last_data['rsi']:.2f} | Stochastic K: {last_data['stoch_k']:.2f} | D: {last_data['stoch_d']:.2f}")
    print(f"  Williams %R: {last_data['willr']:.2f} | CCI: {last_data['cci']:.2f} | MFI: {last_data['mfi']:.2f}")
    osc_signals = {
        'RSI 超买 (>70)': last_signals['rsi_overbought'],
        'RSI 超卖 (<30)': last_signals['rsi_oversold'],
        'RSI 看涨 (50-70)': last_signals['rsi_bullish'],
        'Stochastic 超买': last_signals['stoch_overbought'],
        'Stochastic 超卖': last_signals['stoch_oversold'],
        'Stochastic 金叉': last_signals['stoch_bullish_cross'],
        'Williams %R 超卖': last_signals['willr_oversold'],
        'CCI 看涨': last_signals['cci_bullish'],
        'MFI 看涨': last_signals['mfi_bullish']
    }
    for signal_name, value in osc_signals.items():
        status = "✅" if value else "❌"
        print(f"  {status} {signal_name:<45}")
    
    # ========== 通道指标详情 ==========
    print("\n📉 通道指标:")
    print("-" * 80)
    channel_signals = {
        '布林带买入 (价格<下轨)': last_signals['bb_buy'],
        '布林带卖出 (价格>上轨)': last_signals['bb_sell'],
        '布林带收缩': last_signals['bb_squeeze'],
        '唐奇安买入 (价格>上轨)': last_signals['dc_buy'],
        '唐奇安卖出 (价格<下轨)': last_signals['dc_sell'],
        'Keltner 买入': last_signals['kc_buy'],
        'Keltner 卖出': last_signals['kc_sell']
    }
    for signal_name, value in channel_signals.items():
        status = "✅" if value else "❌"
        print(f"  {status} {signal_name:<45}")
    
    # ========== 成交量指标详情 ==========
    print("\n📊 成交量指标:")
    print("-" * 80)
    volume_ratio = last_data['Volume'] / last_data['volume_sma_20'] if last_data['volume_sma_20'] > 0 else 0
    print(f"  成交量比: {volume_ratio:.2f}x (相对20日均量)")
    volume_signals = {
        '成交量高于平均': last_signals['volume_above_avg'],
        '成交量激增 (>1.5x)': last_signals['volume_spike'],
        'OBV 上升趋势': last_signals['obv_trending_up'],
        '价涨量增 (看涨)': last_signals['volume_price_bullish']
    }
    for signal_name, value in volume_signals.items():
        status = "✅" if value else "❌"
        print(f"  {status} {signal_name:<45}")
    
    # ========== 支撑阻力详情 ==========
    print("\n🎯 支撑阻力:")
    print("-" * 80)
    support_dist = ((last_data['Close'] - last_data['support']) / last_data['support']) * 100
    resistance_dist = ((last_data['resistance'] - last_data['Close']) / last_data['Close']) * 100
    print(f"  支撑位: ${last_data['support']:,.2f} (距离: {support_dist:.2f}%)")
    print(f"  阻力位: ${last_data['resistance']:,.2f} (距离: {resistance_dist:.2f}%)")
    support_resistance_signals = {
        '接近支撑位 (<2%)': last_signals['price_near_support'],
        '接近阻力位 (<2%)': last_signals['price_near_resistance'],
        '突破阻力位': last_signals['price_break_resistance'],
        '跌破支撑位': last_signals['price_break_support']
    }
    for signal_name, value in support_resistance_signals.items():
        status = "✅" if value else "❌"
        print(f"  {status} {signal_name:<45}")
    
    # ========== 动量指标详情 ==========
    print("\n⚡ 动量指标:")
    print("-" * 80)
    print(f"  ROC: {last_data['roc']:.2f}% | MOM: {last_data['mom']:.2f}")
    momentum_signals = {
        'ROC 为正': last_signals['roc_positive'],
        'ROC 上升': last_signals['roc_increasing'],
        'MOM 为正': last_signals['mom_positive'],
        'MOM 上升': last_signals['mom_increasing']
    }
    for signal_name, value in momentum_signals.items():
        status = "✅" if value else "❌"
        print(f"  {status} {signal_name:<45}")
    
    # ========== 波动率详情 ==========
    print("\n📊 波动率:")
    print("-" * 80)
    atr_percent = (last_data['atr'] / last_data['Close']) * 100 if last_data['Close'] > 0 else 0
    print(f"  ATR: {last_data['atr']:.2f} ({atr_percent:.2f}%)")
    volatility_signals = {
        '高波动率': last_signals['atr_high_volatility'],
        '低波动率': last_signals['atr_low_volatility']
    }
    for signal_name, value in volatility_signals.items():
        status = "✅" if value else "❌"
        print(f"  {status} {signal_name:<45}")
    
    print("\n" + "=" * 80)
    
    # 返回报告摘要
    return {
        'timestamp': current_time,
        'price': current_price,
        'recommendation': recommendation,
        'signal_strength': signal_strength,
        'bullish_count': bullish_count,
        'bearish_count': bearish_count
    }


# 显示信号为 DataFrame 表格格式（可选）
def display_signals_table(signals, data):
    """以表格形式显示信号"""
    last_signals = signals.iloc[-1]
    
    # 创建分类的信号表格
    signal_data = {
        '指标类型': [],
        '信号名称': [],
        '状态': [],
        '值': []
    }
    
    # MACD
    signal_data['指标类型'].extend(['MACD', 'MACD'])
    signal_data['信号名称'].extend(['买入信号 (MACD > Signal)', '卖出信号 (MACD < Signal)'])
    signal_data['状态'].extend(['✅ 是' if last_signals['macd_buy'] else '❌ 否', 
                                '✅ 是' if last_signals['macd_sell'] else '❌ 否'])
    signal_data['值'].extend([last_signals['macd_buy'], last_signals['macd_sell']])
    
    # RSI
    rsi_value = data['rsi'].iloc[-1]
    signal_data['指标类型'].extend(['RSI', 'RSI', 'RSI', 'RSI'])
    signal_data['信号名称'].extend([f'RSI 值: {rsi_value:.2f}', '超买 (>70)', '超卖 (<30)', '中性区间 (30-70)'])
    signal_data['状态'].extend(['', 
                                '✅ 是' if last_signals['rsi_overbought'] else '❌ 否',
                                '✅ 是' if last_signals['rsi_oversold'] else '❌ 否',
                                '✅ 是' if last_signals['rsi_middle'] else '❌ 否'])
    signal_data['值'].extend([rsi_value, last_signals['rsi_overbought'], 
                             last_signals['rsi_oversold'], last_signals['rsi_middle']])
    
    # 布林带
    signal_data['指标类型'].extend(['布林带', '布林带', '布林带', '布林带'])
    signal_data['信号名称'].extend(['买入信号 (价格 < 下轨)', '卖出信号 (价格 > 上轨)', 
                                    '买入信号 (价格 > 中轨)', '卖出信号 (价格 < 中轨)'])
    signal_data['状态'].extend(['✅ 是' if last_signals['bollinger_buy'] else '❌ 否',
                                '✅ 是' if last_signals['bollinger_sell'] else '❌ 否',
                                '✅ 是' if last_signals['bollinger_middle_line_buy'] else '❌ 否',
                                '✅ 是' if last_signals['bollinger_middle_line_sell'] else '❌ 否'])
    signal_data['值'].extend([last_signals['bollinger_buy'], last_signals['bollinger_sell'],
                             last_signals['bollinger_middle_line_buy'], last_signals['bollinger_middle_line_sell']])
    
    # 唐奇安通道
    signal_data['指标类型'].extend(['唐奇安通道', '唐奇安通道', '唐奇安通道', '唐奇安通道'])
    signal_data['信号名称'].extend(['买入信号 (价格 > 上轨)', '卖出信号 (价格 < 下轨)', 
                                    '买入信号 (中间)', '卖出信号 (中间)'])
    signal_data['状态'].extend(['✅ 是' if last_signals['donchian_buy'] else '❌ 否',
                                '✅ 是' if last_signals['donchian_sell'] else '❌ 否',
                                '✅ 是' if last_signals['donchian_buy_middle'] else '❌ 否',
                                '✅ 是' if last_signals['donchian_sell_middle'] else '❌ 否'])
    signal_data['值'].extend([last_signals['donchian_buy'], last_signals['donchian_sell'],
                             last_signals['donchian_buy_middle'], last_signals['donchian_sell_middle']])
    
    df_table = pd.DataFrame(signal_data)
    
    # 设置显示选项
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 50)
    
    print(f"\n📊 信号表格 (时间: {signals.index[-1]}, 价格: ${data['Close'].iloc[-1]:,.2f})\n")
    display(df_table)
    
    return df_table

# 使用示例：显示表格格式的信号
# display_signals_table(signals, data)


In [63]:
# ============================================
# 使用示例：完整的技术分析流程
# ============================================

# 1. 设置交易对和时间周期
symbol = 'BTC/USDT'  # 交易对
timeframe = '15m'      # 时间周期：1m, 5m, 15m, 1h, 4h, 1d 等
limit = 599          # 获取的K线数量

# 2. 获取K线数据
print("📥 正在获取数据...")
data = fetch_data(symbol, timeframe=timeframe, limit=limit)

# 3. 计算技术指标
print("📊 正在计算技术指标...")
data = generate_signals(data)

# 4. 分析交易信号
print("🔍 正在分析交易信号...")
signals = analyze_signals(data)

# 5. 显示完整分析报告（推荐）
print("\n" + "="*80)
print("📋 完整分析报告")
print("="*80 + "\n")
report = display_analysis_report(signals, data, show_details=True)

# 6. 可选：显示简要报告（仅显示摘要）
# report_brief = display_analysis_report(signals, data, show_details=False)
# print(f"\n简要摘要: {report_brief}")

# 7. 可选：显示表格格式的信号
# display_signals_table(signals, data)

# 8. 可选：查看原始数据（前5行和后5行）
# print("\n原始数据预览:")
# print(data.head())
# print("\n最新数据:")
# print(data.tail())


📥 正在获取数据...
📊 正在计算技术指标...
🔍 正在分析交易信号...

📋 完整分析报告

📊 技术分析信号汇总 - 综合报告
⏰ 分析时间: 2025-11-05 09:15:00
💰 当前价格: $101,841.92
📈 多头信号数: 7 | 📉 空头信号数: 3
⚡ 综合信号强度: 4
🎯 综合建议: 🟢🟢 买入

📈 趋势指标:
--------------------------------------------------------------------------------
  ❌ MACD 买入                                      
  ❌ MACD 金叉                                      
  ❌ SMA 金叉 (5>20)                                
  ❌ EMA 金叉 (12>26)                               
  ✅ 价格 > SMA50                                   
  ❌ 价格 > SMA200                                  
  ❌ SMA50 > SMA200 (长期趋势)                        
  ❌ ADX 强趋势 (>25)                                
  ❌ ADX 看涨                                       

📊 震荡指标:
--------------------------------------------------------------------------------
  RSI: 52.51 | Stochastic K: 63.18 | D: 49.60
  Williams %R: -35.21 | CCI: 20.75 | MFI: 58.39
  ❌ RSI 超买 (>70)                                 
  ❌ RSI 超卖 (<30)                                 
  ✅ RSI 看

/var/folders/3q/wv1kfqy544v7lnd3d_75mg3w0000gn/T/ipykernel_11856/3965410995.py:214: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  signals['bollinger_middle_line_sell'] = signals['bb_price_below_middle']
/var/folders/3q/wv1kfqy544v7lnd3d_75mg3w0000gn/T/ipykernel_11856/3965410995.py:215: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  signals['donchian_buy'] = signals['dc_buy']
/var/folders/3q/wv1kfqy544v7lnd3d_75mg3w0000gn/T/ipykernel_11856/3965410995.py:216: PerformanceWarning: DataFrame is highly fragmented.  This is usually the 

In [64]:
# =============================================================
# 指标与策略扩展库（按你的分类整理）
# 提供 calculate_* 与 *_strategy，并注册到目录中统一调用
# 依赖: pandas as pd, talib as talib（已在前文导入）
# =============================================================
import numpy as np

# -----------------------------
# Helper 通用函数
# -----------------------------
def _ema(series: pd.Series, period: int) -> pd.Series:
    return series.ewm(span=period, adjust=False).mean()

def _sma(series: pd.Series, period: int) -> pd.Series:
    return series.rolling(window=period, min_periods=period).mean()

def _std(series: pd.Series, period: int) -> pd.Series:
    return series.rolling(window=period, min_periods=period).std()

# 典型价格
def calculate_typical_price(data: pd.DataFrame) -> pd.Series:
    return (data['High'] + data['Low'] + data['Close']) / 3.0

# -----------------------------
# Trend Indicators
# -----------------------------

def calculate_absolute_price_oscillator(data: pd.DataFrame, fast=12, slow=26) -> pd.Series:
    return talib.APO(data['Close'], fastperiod=fast, slowperiod=slow)

def calculate_aroon(data: pd.DataFrame, period=25):
    aroon_down, aroon_up = talib.AROON(data['High'], data['Low'], timeperiod=period)
    return aroon_up, aroon_down

def calculate_balance_of_power(data: pd.DataFrame) -> pd.Series:
    return talib.BOP(data['Open'], data['High'], data['Low'], data['Close'])

def calculate_chande_forecast_oscillator(data: pd.DataFrame, period=9) -> pd.Series:
    # talib 不直接提供 CFO，使用回归预测近似
    tp = calculate_typical_price(data)
    # 线性回归预测值
    lr = talib.LINEARREG(tp, timeperiod=period)
    return ((tp - lr) / tp.replace(0, np.nan)) * 100

def calculate_commodity_channel_index(data: pd.DataFrame, period=20) -> pd.Series:
    return talib.CCI(data['High'], data['Low'], data['Close'], timeperiod=period)

def calculate_double_exponential_moving_average(data: pd.DataFrame, period=30) -> pd.Series:
    return talib.DEMA(data['Close'], timeperiod=period)

def calculate_exponential_moving_average(data: pd.DataFrame, period=20) -> pd.Series:
    return talib.EMA(data['Close'], timeperiod=period)

def calculate_mass_index(data: pd.DataFrame, period=25) -> pd.Series:
    # Range EMA 与其 EMA 之比的累积
    rng = data['High'] - data['Low']
    ema1 = _ema(rng, 9)
    ema2 = _ema(ema1, 9)
    mi = (ema1 / ema2).rolling(window=period, min_periods=period).sum()
    return mi

def calculate_moving_average_convergence_divergence(data: pd.DataFrame, fast=12, slow=26, signal=9):
    macd, macd_signal, macd_hist = talib.MACD(data['Close'], fastperiod=fast, slowperiod=slow, signalperiod=signal)
    return macd, macd_signal, macd_hist

def calculate_moving_max(data: pd.DataFrame, period=20) -> pd.Series:
    return talib.MAX(data['Close'], timeperiod=period)

def calculate_moving_min(data: pd.DataFrame, period=20) -> pd.Series:
    return talib.MIN(data['Close'], timeperiod=period)

def calculate_moving_sum(data: pd.DataFrame, period=20) -> pd.Series:
    return talib.SUM(data['Close'], timeperiod=period)

def calculate_parabolic_sar(data: pd.DataFrame, accel=0.02, maximum=0.2) -> pd.Series:
    return talib.SAR(data['High'], data['Low'], acceleration=accel, maximum=maximum)

def calculate_qstick(data: pd.DataFrame, period=14) -> pd.Series:
    # (Close - Open) 的移动平均
    return _sma(data['Close'] - data['Open'], period)

def calculate_kdj(data: pd.DataFrame, k=14, d=3, j=3):
    k_fast, d_slow = talib.STOCH(data['High'], data['Low'], data['Close'], fastk_period=k, slowk_period=d, slowd_period=d)
    j_line = 3 * k_fast - 2 * d_slow
    return k_fast, d_slow, j_line

def calculate_rolling_moving_average(data: pd.DataFrame, period=14) -> pd.Series:
    # RMA = Wilder 的 EMA
    alpha = 1.0 / period
    return data['Close'].ewm(alpha=alpha, adjust=False).mean()

def calculate_simple_moving_average(data: pd.DataFrame, period=20) -> pd.Series:
    return talib.SMA(data['Close'], timeperiod=period)

def calculate_since_change(data: pd.DataFrame, threshold=0.02) -> pd.Series:
    # 距离上次 |变化幅度| > threshold 的bar数
    pct = data['Close'].pct_change().abs()
    idx = pct.gt(threshold)
    counter = np.zeros(len(pct), dtype=int)
    last = -1
    for i, flag in enumerate(idx):
        if flag:
            last = i
        counter[i] = 0 if last == -1 else i - last
    return pd.Series(counter, index=data.index)

def calculate_triple_exponential_moving_average(data: pd.DataFrame, period=30) -> pd.Series:
    return talib.TEMA(data['Close'], timeperiod=period)

def calculate_triangular_moving_average(data: pd.DataFrame, period=30) -> pd.Series:
    return talib.TRIMA(data['Close'], timeperiod=period)

def calculate_triple_exponential_average(data: pd.DataFrame, period=15) -> pd.Series:
    # TRIX = 1-period % change of triple-smoothed EMA
    trix = talib.TRIX(data['Close'], timeperiod=period)
    return trix

def calculate_volume_weighted_moving_average(data: pd.DataFrame, period=20) -> pd.Series:
    tp = calculate_typical_price(data)
    vwma = (tp * data['Volume']).rolling(period, min_periods=period).sum() / data['Volume'].rolling(period, min_periods=period).sum()
    return vwma

def calculate_vortex(data: pd.DataFrame, period=14):
    # Vortex 指标：VI+ 与 VI-
    tr = talib.TRANGE(data['High'], data['Low'], data['Close'])
    vm_plus = (data['High'] - data['Low'].shift(1)).abs()
    vm_minus = (data['Low'] - data['High'].shift(1)).abs()
    vi_plus = vm_plus.rolling(period).sum() / tr.rolling(period).sum()
    vi_minus = vm_minus.rolling(period).sum() / tr.rolling(period).sum()
    return vi_plus, vi_minus

# -----------------------------
# Momentum Indicators
# -----------------------------

def calculate_awesome_oscillator(data: pd.DataFrame, fast=5, slow=34) -> pd.Series:
    median = (data['High'] + data['Low']) / 2
    return _sma(median, fast) - _sma(median, slow)

def calculate_chaikin_oscillator(data: pd.DataFrame, fast=3, slow=10) -> pd.Series:
    ad = talib.AD(data['High'], data['Low'], data['Close'], data['Volume'])
    return _ema(ad, fast) - _ema(ad, slow)

def calculate_ichimoku_cloud(data: pd.DataFrame):
    # 转换线(9), 基准线(26), 先行A, 先行B, 滞后(26)
    conv = (data['High'].rolling(9).max() + data['Low'].rolling(9).min()) / 2
    base = (data['High'].rolling(26).max() + data['Low'].rolling(26).min()) / 2
    span_a = ((conv + base) / 2).shift(26)
    span_b = ((data['High'].rolling(52).max() + data['Low'].rolling(52).min()) / 2).shift(26)
    lag = data['Close'].shift(-26)
    return conv, base, span_a, span_b, lag

def calculate_percentage_price_oscillator(data: pd.DataFrame, fast=12, slow=26) -> pd.Series:
    return talib.PPO(data['Close'], fastperiod=fast, slowperiod=slow)

def calculate_percentage_volume_oscillator(data: pd.DataFrame, fast=12, slow=26) -> pd.Series:
    fast_ema = _ema(data['Volume'], fast)
    slow_ema = _ema(data['Volume'], slow)
    return (fast_ema - slow_ema) / slow_ema.replace(0, np.nan) * 100

def calculate_price_rate_of_change(data: pd.DataFrame, period=10) -> pd.Series:
    return talib.ROC(data['Close'], timeperiod=period)

def calculate_relative_strength_index(data: pd.DataFrame, period=14) -> pd.Series:
    return talib.RSI(data['Close'], timeperiod=period)

def calculate_stochastic_oscillator(data: pd.DataFrame, k=14, d=3):
    k_fast, d_slow = talib.STOCH(data['High'], data['Low'], data['Close'], fastk_period=k, slowk_period=d, slowd_period=d)
    return k_fast, d_slow

def calculate_williams_r(data: pd.DataFrame, period=14) -> pd.Series:
    return talib.WILLR(data['High'], data['Low'], data['Close'], timeperiod=period)

# -----------------------------
# Volatility Indicators
# -----------------------------

def calculate_acceleration_bands(data: pd.DataFrame, period=20, factor=4):
    tp = calculate_typical_price(data)
    hl = data['High'] - data['Low']
    upper = data['High'] * (1 + factor * hl / tp.replace(0, np.nan))
    lower = data['Low'] * (1 - factor * hl / tp.replace(0, np.nan))
    upper = upper.rolling(period).mean()
    lower = lower.rolling(period).mean()
    middle = _sma(tp, period)
    return upper, middle, lower

def calculate_average_true_range(data: pd.DataFrame, period=14) -> pd.Series:
    return talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=period)

def calculate_bollinger_bands(data: pd.DataFrame, period=20, nbdev=2):
    upper, middle, lower = talib.BBANDS(data['Close'], timeperiod=period, nbdevup=nbdev, nbdevdn=nbdev, matype=0)
    return upper, middle, lower

def calculate_bollinger_bands_width(data: pd.DataFrame, period=20, nbdev=2) -> pd.Series:
    upper, middle, lower = calculate_bollinger_bands(data, period, nbdev)
    return (upper - lower) / middle.replace(0, np.nan) * 100

def calculate_chandelier_exit(data: pd.DataFrame, period=22, atr_mult=3):
    atr = calculate_average_true_range(data, period)
    long_exit = data['High'].rolling(period).max() - atr_mult * atr
    short_exit = data['Low'].rolling(period).min() + atr_mult * atr
    return long_exit, short_exit

def calculate_donchian_channel(data: pd.DataFrame, period=20):
    upper = data['High'].rolling(period).max()
    lower = data['Low'].rolling(period).min()
    middle = (upper + lower) / 2
    return upper, middle, lower

def calculate_keltner_channel(data: pd.DataFrame, period=20, mult=2):
    ema = talib.EMA(data['Close'], timeperiod=period)
    atr = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=period)
    upper = ema + mult * atr
    lower = ema - mult * atr
    return upper, ema, lower

def calculate_moving_standard_deviation(data: pd.DataFrame, period=20) -> pd.Series:
    return talib.STDDEV(data['Close'], timeperiod=period, nbdev=1)

def calculate_projection_oscillator(data: pd.DataFrame, period=14) -> pd.Series:
    # 近似实现：对 Close 做线性回归投影，计算相对位置
    lr = talib.LINEARREG(data['Close'], timeperiod=period)
    lr_slope = talib.LINEARREG_SLOPE(data['Close'], timeperiod=period)
    proj = lr + lr_slope * np.arange(len(data))
    proj = pd.Series(proj, index=data.index)
    hi = data['High'].rolling(period).max()
    lo = data['Low'].rolling(period).min()
    return (data['Close'] - lo) / (hi - lo).replace(0, np.nan) * 100

def calculate_true_range(data: pd.DataFrame) -> pd.Series:
    return talib.TRANGE(data['High'], data['Low'], data['Close'])

def calculate_ulcer_index(data: pd.DataFrame, period=14) -> pd.Series:
    # UI: 过去N日的跌幅平方均值的平方根
    dd = (data['Close'] / data['Close'].rolling(period).max() - 1.0) * 100
    return (dd.pow(2).rolling(period).mean()).pow(0.5)

# -----------------------------
# Volume Indicators
# -----------------------------

def calculate_accumulation_distribution(data: pd.DataFrame) -> pd.Series:
    return talib.AD(data['High'], data['Low'], data['Close'], data['Volume'])

def calculate_chaikin_money_flow(data: pd.DataFrame, period=20) -> pd.Series:
    mfm = ((data['Close'] - data['Low']) - (data['High'] - data['Close'])) / (data['High'] - data['Low']).replace(0, np.nan)
    mfv = mfm * data['Volume']
    return mfv.rolling(period).sum() / data['Volume'].rolling(period).sum()

def calculate_ease_of_movement(data: pd.DataFrame, period=14) -> pd.Series:
    mid_move = ((data['High'] + data['Low']) / 2.0).diff()
    br = data['High'] - data['Low']
    emv = mid_move * (data['High'] - data['Low']) / br.replace(0, np.nan)
    return _sma(emv, period)

def calculate_force_index(data: pd.DataFrame, period=13) -> pd.Series:
    fi = data['Close'].diff() * data['Volume']
    return _ema(fi, period)

def calculate_money_flow_index(data: pd.DataFrame, period=14) -> pd.Series:
    return talib.MFI(data['High'], data['Low'], data['Close'], data['Volume'], timeperiod=period)

def calculate_negative_volume_index(data: pd.DataFrame) -> pd.Series:
    nvi = pd.Series(index=data.index, dtype=float)
    nvi.iloc[0] = 1000.0
    for i in range(1, len(data)):
        if data['Volume'].iloc[i] < data['Volume'].iloc[i-1]:
            pct = data['Close'].pct_change().iloc[i]
            nvi.iloc[i] = nvi.iloc[i-1] * (1 + pct)
        else:
            nvi.iloc[i] = nvi.iloc[i-1]
    return nvi

def calculate_on_balance_volume(data: pd.DataFrame) -> pd.Series:
    return talib.OBV(data['Close'], data['Volume'])

def calculate_volume_price_trend(data: pd.DataFrame) -> pd.Series:
    pct = data['Close'].pct_change().fillna(0)
    vpt = (pct * data['Volume']).cumsum()
    return vpt

def calculate_volume_weighted_average_price(data: pd.DataFrame) -> pd.Series:
    # VWAP 通常用于日内，这里按全样本累积近似
    tp = calculate_typical_price(data)
    cum_v = data['Volume'].cumsum().replace(0, np.nan)
    return (tp * data['Volume']).cumsum() / cum_v

# -----------------------------
# 策略（示例，返回布尔信号Series）
# -----------------------------

def calculate_absolute_price_oscillator_strategy(data: pd.DataFrame, fast=12, slow=26):
    apo = calculate_absolute_price_oscillator(data, fast, slow)
    buy = apo > 0
    sell = apo < 0
    return buy, sell

def calculate_aroon_strategy(data: pd.DataFrame, period=25):
    up, down = calculate_aroon(data, period)
    buy = (up > down) & (up.shift(1) <= down.shift(1))
    sell = (up < down) & (up.shift(1) >= down.shift(1))
    return buy, sell

def calculate_balance_of_power_strategy(data: pd.DataFrame, threshold=0):
    bop = calculate_balance_of_power(data)
    return bop > threshold, bop < -threshold

def calculate_chande_forecast_oscillator_strategy(data: pd.DataFrame, period=9, up=5, dn=-5):
    cfo = calculate_chande_forecast_oscillator(data, period)
    return cfo > up, cfo < dn

def calculate_kdj_strategy(data: pd.DataFrame, k=14, d=3):
    kf, ds, j = calculate_kdj(data, k, d)
    return (kf > ds) & (kf.shift(1) <= ds.shift(1)), (kf < ds) & (kf.shift(1) >= ds.shift(1))

def calculate_macd_strategy(data: pd.DataFrame, fast=12, slow=26, signal=9):
    macd, sig, _ = calculate_moving_average_convergence_divergence(data, fast, slow, signal)
    return (macd > sig) & (macd.shift(1) <= sig.shift(1)), (macd < sig) & (macd.shift(1) >= sig.shift(1))

def calculate_parabolic_sar_strategy(data: pd.DataFrame, accel=0.02, maximum=0.2):
    psar = calculate_parabolic_sar(data, accel, maximum)
    return data['Close'] > psar, data['Close'] < psar

def calculate_typical_price_strategy(data: pd.DataFrame, period=20):
    tp = calculate_typical_price(data)
    sma = _sma(tp, period)
    return tp > sma, tp < sma

def calculate_volume_weighted_moving_average_strategy(data: pd.DataFrame, period=20):
    vwma = calculate_volume_weighted_moving_average(data, period)
    return data['Close'] > vwma, data['Close'] < vwma

def calculate_vortex_strategy(data: pd.DataFrame, period=14):
    vi_plus, vi_minus = calculate_vortex(data, period)
    return (vi_plus > vi_minus) & (vi_plus.shift(1) <= vi_minus.shift(1)), (vi_plus < vi_minus) & (vi_plus.shift(1) >= vi_minus.shift(1))

def calculate_momentum_strategy(data: pd.DataFrame, period=10):
    roc = calculate_price_rate_of_change(data, period)
    return roc > 0, roc < 0

def calculate_awesome_oscillator_strategy(data: pd.DataFrame):
    ao = calculate_awesome_oscillator(data)
    return ao > 0, ao < 0

def calculate_ichimoku_cloud_strategy(data: pd.DataFrame):
    conv, base, span_a, span_b, _ = calculate_ichimoku_cloud(data)
    buy = (data['Close'] > span_a) & (data['Close'] > span_b) & (conv > base)
    sell = (data['Close'] < span_a) & (data['Close'] < span_b) & (conv < base)
    return buy, sell

def calculate_rsi2_strategy(data: pd.DataFrame, period=2, ob=90, os=10):
    rsi2 = talib.RSI(data['Close'], timeperiod=period)
    return rsi2 < os, rsi2 > ob

def calculate_stochastic_oscillator_strategy(data: pd.DataFrame, k=14, d=3):
    kf, ds = calculate_stochastic_oscillator(data, k, d)
    return (kf > ds) & (kf.shift(1) <= ds.shift(1)), (kf < ds) & (kf.shift(1) >= ds.shift(1))

def calculate_williams_r_strategy(data: pd.DataFrame, period=14, ob=-20, os=-80):
    wr = calculate_williams_r(data, period)
    return wr < os, wr > ob

def calculate_acceleration_bands_strategy(data: pd.DataFrame, period=20, factor=4):
    up, mid, low = calculate_acceleration_bands(data, period, factor)
    return data['Close'] > up, data['Close'] < low

def calculate_bollinger_bands_strategy(data: pd.DataFrame, period=20, nbdev=2):
    up, mid, low = calculate_bollinger_bands(data, period, nbdev)
    return data['Close'] < low, data['Close'] > up

def calculate_projection_oscillator_strategy(data: pd.DataFrame, period=14, up=80, dn=20):
    po = calculate_projection_oscillator(data, period)
    return po > up, po < dn

def calculate_chaikin_money_flow_strategy(data: pd.DataFrame, period=20, th=0):
    cmf = calculate_chaikin_money_flow(data, period)
    return cmf > th, cmf < -th

def calculate_ease_of_movement_strategy(data: pd.DataFrame, period=14, th=0):
    emv = calculate_ease_of_movement(data, period)
    return emv > th, emv < -th

def calculate_force_index_strategy(data: pd.DataFrame, period=13, th=0):
    fi = calculate_force_index(data, period)
    return fi > th, fi < -th

def calculate_money_flow_index_strategy(data: pd.DataFrame, period=14, ob=80, os=20):
    mfi = calculate_money_flow_index(data, period)
    return mfi < os, mfi > ob

def calculate_negative_volume_index_strategy(data: pd.DataFrame, th_slope=0):
    nvi = calculate_negative_volume_index(data)
    slope = nvi.diff()
    return slope > th_slope, slope < -th_slope

def calculate_volume_weighted_average_price_strategy(data: pd.DataFrame, lookback=50):
    vwap = calculate_volume_weighted_average_price(data)
    ma = _sma(vwap, lookback)
    return vwap > ma, vwap < ma

# -----------------------------
# 注册目录：便于按类批量计算
# -----------------------------
TREND_INDICATORS = {
    'APO': calculate_absolute_price_oscillator,
    'Aroon': calculate_aroon,
    'BOP': calculate_balance_of_power,
    'CFO': calculate_chande_forecast_oscillator,
    'CCI': calculate_commodity_channel_index,
    'DEMA': calculate_double_exponential_moving_average,
    'EMA': calculate_exponential_moving_average,
    'MassIndex': calculate_mass_index,
    'MACD': calculate_moving_average_convergence_divergence,
    'MMAX': calculate_moving_max,
    'MMIN': calculate_moving_min,
    'MSUM': calculate_moving_sum,
    'PSAR': calculate_parabolic_sar,
    'Qstick': calculate_qstick,
    'KDJ': calculate_kdj,
    'RMA': calculate_rolling_moving_average,
    'SMA': calculate_simple_moving_average,
    'SinceChange': calculate_since_change,
    'TEMA': calculate_triple_exponential_moving_average,
    'TRIMA': calculate_triangular_moving_average,
    'TRIX': calculate_triple_exponential_average,
    'TypicalPrice': calculate_typical_price,
    'VWMA': calculate_volume_weighted_moving_average,
    'Vortex': calculate_vortex,
}

MOMENTUM_INDICATORS = {
    'AO': calculate_awesome_oscillator,
    'ChaikinOsc': calculate_chaikin_oscillator,
    'Ichimoku': calculate_ichimoku_cloud,
    'PPO': calculate_percentage_price_oscillator,
    'PVO': calculate_percentage_volume_oscillator,
    'ROC': calculate_price_rate_of_change,
    'RSI': calculate_relative_strength_index,
    'STOCH': calculate_stochastic_oscillator,
    'WILLR': calculate_williams_r,
}

VOLATILITY_INDICATORS = {
    'AB': calculate_acceleration_bands,
    'ATR': calculate_average_true_range,
    'BB': calculate_bollinger_bands,
    'BBW': calculate_bollinger_bands_width,
    'CE': calculate_chandelier_exit,
    'DC': calculate_donchian_channel,
    'KC': calculate_keltner_channel,
    'MSTD': calculate_moving_standard_deviation,
    'PO': calculate_projection_oscillator,
    'TR': calculate_true_range,
    'UI': calculate_ulcer_index,
}

VOLUME_INDICATORS = {
    'AD': calculate_accumulation_distribution,
    'CMF': calculate_chaikin_money_flow,
    'EMV': calculate_ease_of_movement,
    'FI': calculate_force_index,
    'MFI': calculate_money_flow_index,
    'NVI': calculate_negative_volume_index,
    'OBV': calculate_on_balance_volume,
    'VPT': calculate_volume_price_trend,
    'VWAP': calculate_volume_weighted_average_price,
}

# 策略注册
TREND_STRATEGIES = {
    'APO_Strategy': calculate_absolute_price_oscillator_strategy,
    'Aroon_Strategy': calculate_aroon_strategy,
    'BOP_Strategy': calculate_balance_of_power_strategy,
    'CFO_Strategy': calculate_chande_forecast_oscillator_strategy,
    'KDJ_Strategy': calculate_kdj_strategy,
    'MACD_Strategy': calculate_macd_strategy,
    'PSAR_Strategy': calculate_parabolic_sar_strategy,
    'TypicalPrice_Strategy': calculate_typical_price_strategy,
    'VWMA_Strategy': calculate_volume_weighted_moving_average_strategy,
    'Vortex_Strategy': calculate_vortex_strategy,
}

MOMENTUM_STRATEGIES = {
    'Momentum_Strategy': calculate_momentum_strategy,
    'AO_Strategy': calculate_awesome_oscillator_strategy,
    'Ichimoku_Strategy': calculate_ichimoku_cloud_strategy,
    'RSI2_Strategy': calculate_rsi2_strategy,
    'STOCH_Strategy': calculate_stochastic_oscillator_strategy,
    'WILLR_Strategy': calculate_williams_r_strategy,
}

VOLATILITY_STRATEGIES = {
    'AB_Strategy': calculate_acceleration_bands_strategy,
    'BB_Strategy': calculate_bollinger_bands_strategy,
    'PO_Strategy': calculate_projection_oscillator_strategy,
}

VOLUME_STRATEGIES = {
    'CMF_Strategy': calculate_chaikin_money_flow_strategy,
    'EMV_Strategy': calculate_ease_of_movement_strategy,
    'FI_Strategy': calculate_force_index_strategy,
    'MFI_Strategy': calculate_money_flow_index_strategy,
    'NVI_Strategy': calculate_negative_volume_index_strategy,
    'VWAP_Strategy': calculate_volume_weighted_average_price_strategy,
}

# -----------------------------
# 批量计算与汇总
# -----------------------------

def compute_selected_indicators(data: pd.DataFrame,
                               trend=True, momentum=True, volatility=True, volume=True) -> dict:
    results = {}
    if trend:
        results['trend'] = {name: func(data) for name, func in TREND_INDICATORS.items()}
    if momentum:
        results['momentum'] = {name: func(data) for name, func in MOMENTUM_INDICATORS.items()}
    if volatility:
        results['volatility'] = {name: func(data) for name, func in VOLATILITY_INDICATORS.items()}
    if volume:
        results['volume'] = {name: func(data) for name, func in VOLUME_INDICATORS.items()}
    return results


def compute_selected_strategies(data: pd.DataFrame,
                               trend=True, momentum=True, volatility=True, volume=True) -> dict:
    signals = {}
    if trend:
        signals['trend'] = {name: func(data) for name, func in TREND_STRATEGIES.items()}
    if momentum:
        signals['momentum'] = {name: func(data) for name, func in MOMENTUM_STRATEGIES.items()}
    if volatility:
        signals['volatility'] = {name: func(data) for name, func in VOLATILITY_STRATEGIES.items()}
    if volume:
        signals['volume'] = {name: func(data) for name, func in VOLUME_STRATEGIES.items()}
    return signals


# 使用示例（可选）
# indicators = compute_selected_indicators(data)
# strategies = compute_selected_strategies(data)
# 某策略信号: buy_sig, sell_sig = strategies['trend']['MACD_Strategy'](data)

